# Testing Nsaba Functionality

##### Current Methods
- get_aba_ge()
- ge_ratio()
- get_ns_act()
- make_ge_ns_mat()
- coords_to_ge()

In [3]:
%matplotlib inline
from nsaba.nsaba import Nsaba
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import itertools

In [4]:
%load_ext line_profiler

In [2]:
# Simon Path IO

data_dir = '../../data_dir'
os.chdir(data_dir)
Nsaba.aba_load()
Nsaba.ns_load()

This may take a minute or two ...
SampleAnnot.csv loaded.
MicroarrayExpression.csv loaded.
Probes.csv loaded.
Nsaba.aba['mni_coords'] initialized.

This may take a minute or two ...
database.txt loaded.
features.txt loaded.
Nsaba.ns['mni_coords'] initialized.



In [ ]:
#Torben Path IO

ns_path = "/Users/Torben/Documents/ABI analysis/current_data_new/"
aba_path = '/Users/Torben/Documents/ABI analysis/normalized_microarray_donor9861/'
#ns_path = '/Users/SP/Documents/'
#aba_path = '/Users/SP/Documents/'

Nsaba.aba_load(aba_path)
Nsaba.ns_load(ns_path)

In [ ]:
# Loading gene expression for all ABA registered Entrez IDs.
A = Nsaba()
A.load_ge_pickle('Nsaba_ABA_ge.pkl')
%time A.get_ns_act('attention', thresh = -1)

In [5]:
# Testing ge_ratio()
A = Nsaba()
A.ge_ratio((1813,1816))

array([ 0.57446139,  0.99023239,  1.11575466,  1.01100606,  0.90041432,
        0.51675821,  0.60023131,  0.79145099,  0.49431709,  0.48482013,
        0.69057069,  0.81420569,  0.47455972,  0.84989612,  0.62712447,
        0.63073593,  0.53755516,  0.44417201,  0.58291914,  0.442529  ,
        0.57296542,  0.66765314,  0.78020594,  0.60869062,  1.17611206,
        1.2758823 ,  0.6127578 ,  0.57277835,  1.10183741,  0.5111504 ,
        0.54529738,  0.50251657,  0.7318342 ,  0.94982714,  0.82175693,
        0.74350377,  1.06186932,  1.32446539,  0.45594681,  0.43065786,
        0.5671627 ,  0.59765326,  0.47546101,  0.46028177,  0.5367075 ,
        0.46101267,  0.42851074,  0.54803596,  0.56806979,  0.46818031,
        0.47677465,  0.41901151,  0.57470563,  0.48748218,  0.43258916,
        0.4557625 ,  0.48683465,  0.55563527,  0.63151196,  0.56402208,
        0.46587026,  0.59480653,  0.60864499,  0.50248616,  0.57100629,
        0.51971739,  0.50266858,  0.49048015,  0.5850243 ,  0.59

### Coordinates to gene expression:

Provide a list of coordinates and entrez_ids and the function will return matrix of coordinates by gene expression

In [ ]:
rand = lambda null: np.random.uniform(-10,10,3).tolist()
coord_num = 20
coords = [rand(0) for i in range(coord_num)]

A.coords_to_ge(coords, entrez_ids=[733,88,33], search_radii=8)

In [ ]:
A.get_ns_act("attention", thresh=.35, method='knn')

# You can use the sphere method too, if you want to weight by bucket. 
# e.g:
# A.get_ns_act("attention", thresh=.3, method='sphere')

A.make_ge_ns_mat('attention', [733, 33, 88])

In [ ]:
A.make_ge_ns_mat('spatial', [733, 33, 88])

## Visualization Methods (testing)

In [ ]:
def visualize_ns(term, points=200):
        if term in A.term:
            term_index = A.ns['features_df'].columns.get_loc(term)
            import matplotlib.pyplot as plt
            import matplotlib.cm as cm
            from mpl_toolkits.mplot3d import Axes3D
            rand_point_inds = np.random.random_integers(0, len(np.squeeze(zip(A.ns['mni_coords'].data))), points)
            rand_points = np.squeeze(zip(A.ns['mni_coords'].data))[rand_point_inds]
            weights = []
            inds_of_real_points_with_no_fucking_missing_study_ids = []
            for rand_point in range(len(rand_points)):
                if len(A.coord_to_terms(rand_points[rand_point].astype(list))) > 0:
                    print 'good point' + str(rand_points[rand_point])
                    inds_of_real_points_with_no_fucking_missing_study_ids.append(rand_point_inds[rand_point])
                    weights.append(A.coord_to_terms(rand_points[rand_point].astype(list))[term_index])
                else:
                    print 'bad point ' + str(rand_points[rand_point])
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            colors = cm.jet(weights/max(weights))
            color_map = cm.ScalarMappable(cmap=cm.jet)
            color_map.set_array(weights)
            fig.colorbar(color_map)
            x = A.ns['mni_coords'].data[inds_of_real_points_with_no_fucking_missing_study_ids, 0]
            y = A.ns['mni_coords'].data[inds_of_real_points_with_no_fucking_missing_study_ids, 1]
            z = A.ns['mni_coords'].data[inds_of_real_points_with_no_fucking_missing_study_ids, 2]
        else:
            print 'Term '+term + ' has not been initialized. Use self.get_ns_act(' + term + ',thresh = 0.01)'
        ax.scatter(x, y, z, c=colors, alpha=0.4)
        ax.set_title('Estimation of ' + term)

        return fig
    
visualize_ns('attention')